In [2]:
import pandas as pd
import numpy as np

In [3]:
## load the combined dataset

df = pd.read_csv(r'C:\Users\qfu88\Desktop\KwF\Bioacoustics-analysis\Datas\df_with_sim_phase.csv')

In [42]:
df.head()

,Recorder,Timestamp,Human Activity,Human Activity Score,species,confidence,Sim Type,Sim Relative Time,Temperature,Windspeed,Precipitation,Humidity,Weather Desc,Timestamp UTC,Time Of Day,Datetime,Sim Phase
535767,Audio_Moth_6,2025-03-17 11:52:05,no,0.0,No Bird Detected,0.0,[],[],29.9,5.6,0.0,69,Overcast,2025-03-17 18:00:00+00:00,afternoon,2025-03-17 12:00:00,No Simulation
535768,Audio_Moth_6,2025-03-17 11:52:08,no,0.0,No Bird Detected,0.0,[],[],29.9,5.6,0.0,69,Overcast,2025-03-17 18:00:00+00:00,afternoon,2025-03-17 12:00:00,No Simulation
535769,Audio_Moth_6,2025-03-17 11:52:11,no,0.0,No Bird Detected,0.0,[],[],29.9,5.6,0.0,69,Overcast,2025-03-17 18:00:00+00:00,afternoon,2025-03-17 12:00:00,No Simulation
535770,Audio_Moth_6,2025-03-17 11:52:14,no,0.0,No Bird Detected,0.0,[],[],29.9,5.6,0.0,69,Overcast,2025-03-17 18:00:00+00:00,afternoon,2025-03-17 12:00:00,No Simulation
535771,Audio_Moth_6,2025-03-17 11:52:17,no,0.0,No Bird Detected,0.0,[],[],29.9,5.6,0.0,69,Overcast,2025-03-17 18:00:00+00:00,afternoon,2025-03-17 12:00:00,No Simulation


In [4]:
##then split out the 6 audiomoths data
df_moth1 = df[df['Recorder'] == 'Audio_Moth_1'].copy()
df_moth2 = df[df['Recorder'] == 'Audio_Moth_2'].copy()
df_moth3 = df[df['Recorder'] == 'Audio_Moth_3'].copy()
df_moth4 = df[df['Recorder'] == 'Audio_Moth_4'].copy()
df_moth5 = df[df['Recorder'] == 'Audio_Moth_5'].copy()
df_moth6 = df[df['Recorder'] == 'Audio_Moth_6'].copy()

In [5]:
##sort them into a dic for easy access 
moth_datasets = {
    "Audio_Moth_1": df_moth1,
    "Audio_Moth_2": df_moth2,
    "Audio_Moth_3": df_moth3,
    "Audio_Moth_4": df_moth4,
    "Audio_Moth_5": df_moth5,
    "Audio_Moth_6": df_moth6,
}

In [6]:
#load event block data
df = pd.read_csv(r"C:\Users\qfu88\Desktop\KwF\Bioacoustics-analysis\Datas\six sim block tables\sim_blocks_all.csv")

In [7]:
df.head()

,Start_Time,End_Time,Sim Type Label,Duration_Minutes,Recorder
0,2025-03-18 17:14:18,2025-03-18 17:19:18,Vehicle,5.00,Audio_Moth_1
1,2025-03-19 07:25:54,2025-03-19 07:37:24,Human Presence on Trail,11.50,Audio_Moth_1
2,2025-03-19 07:37:27,2025-03-19 07:42:27,"Human Presence on Trail, Vehicle",5.00,Audio_Moth_1
3,2025-03-19 07:42:30,2025-03-19 07:45:51,Human Presence on Trail,3.35,Audio_Moth_1
4,2025-03-19 17:14:39,2025-03-19 17:25:33,Human Presence on Trail,10.90,Audio_Moth_1


In [8]:
# fileter out only vehicle, so we'll have a table of only vehicle related events across all audiomoths 

vehicle_block = df[df['Sim Type Label'].str.contains("Vehicle", case=False)].copy()

vehicle_block

,Start_Time,End_Time,Sim Type Label,Duration_Minutes,Recorder
0,2025-03-18 17:14:18,2025-03-18 17:19:18,Vehicle,5.00,Audio_Moth_1
2,2025-03-19 07:37:27,2025-03-19 07:42:27,"Human Presence on Trail, Vehicle",5.00,Audio_Moth_1
5,2025-03-19 17:25:36,2025-03-19 17:30:33,"Human Presence on Trail, Vehicle",4.95,Audio_Moth_1
9,2025-03-19 07:52:43,2025-03-19 07:57:40,"Human Presence on Trail, Vehicle",4.95,Audio_Moth_2
20,2025-03-19 08:39:48,2025-03-19 08:44:48,"Human Presence on Trail, Vehicle",5.00,Audio_Moth_4
24,2025-03-19 16:48:22,2025-03-19 16:53:19,"Human Presence on Trail, Vehicle",4.95,Audio_Moth_4
33,2025-03-21 08:30:39,2025-03-21 08:31:15,"Human Presence on Trail, Vehicle",0.60,Audio_Moth_4
34,2025-03-21 08:31:18,2025-03-21 08:35:36,"Chainsaw, Human Presence on Trail, Vehicle",4.30,Audio_Moth_4
38,2025-03-21 14:01:48,2025-03-21 14:05:39,"Human Presence on Trail, Vehicle",3.85,Audio_Moth_4
46,2025-03-19 08:54:41,2025-03-19 08:59:38,"Human Presence on Trail, Vehicle",4.95,Audio_Moth_5


now from above table we can know that when and where(which recorder) each vehicel related simulation occured
then we can go back to each recorder dataset and extract bird detections in the three time windows: 
before, during, after event. 

In [9]:
## the below function is created to extract the three time windowns for one event, we can later use it for the rest other events 


def extract_vehicle_windows(df, sim_start, sim_end, recorder, sim_label, before_min=15, after_min=15):
    df = df.copy()
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

    # define time range
    win_before = (sim_start - pd.Timedelta(minutes=before_min), sim_start)
    win_during = (sim_start, sim_end)
    win_after = (sim_end, sim_end + pd.Timedelta(minutes=after_min))

    # slice windows
    def slice_window(df, window):
        s, e = window
        return df[(df["Timestamp"] >= s) & (df["Timestamp"] <= e)].copy()

    df_before = slice_window(df, win_before)
    df_during = slice_window(df, win_during)
    df_after = slice_window(df, win_after)

    # add lable and other features 
    for d, label in [(df_before, "before"), (df_during, "during"), (df_after, "after")]:
        d["period"] = label
        d["Recorder"] = recorder
        d["Sim_Type_Label"] = sim_label
        d["Sim_Start"] = sim_start
        d["Sim_End"] = sim_end

    # Combine
    df_window = pd.concat([df_before, df_during, df_after], ignore_index=True)
    return df_window




In [10]:
## now apply above function to all vehicle events 


vehicle_windows_all = []

for _, row in vehicle_block.iterrows():
    recorder = row["Recorder"]
    sim_start = pd.to_datetime(row["Start_Time"])
    sim_end = pd.to_datetime(row["End_Time"])
    sim_label = row["Sim Type Label"]

    # match to datasets
    df = moth_datasets[recorder]


    df_window = extract_vehicle_windows(df, sim_start, sim_end, recorder, sim_label)
    vehicle_windows_all.append(df_window)

vehicle_windows_all = pd.concat(vehicle_windows_all, ignore_index=True)
vehicle_windows_all.to_csv("vehicle_windows_all.csv", index=False)



In [11]:
## let's check out this csv file generaged from above

df_vehicle = pd.read_csv(r"C:\Users\qfu88\Desktop\KwF\Bioacoustics-analysis\Datas\six sim block tables\vehicle_windows_all.csv")

In [12]:
df_vehicle.head()

,Recorder,Timestamp,Human Activity,Human Activity Score,species,confidence,Sim Type,Sim Relative Time,Temperature,Windspeed,...,Humidity,Weather Desc,Timestamp UTC,Time Of Day,Datetime,Sim Phase,period,Sim_Type_Label,Sim_Start,Sim_End
0,Audio_Moth_1,2025-03-18 16:59:18,no,0.0,No Bird Detected,0.000000,[],[],27.3,5.0,...,83,Mainly clear,2025-03-18 23:00:00+00:00,evening,2025-03-18 17:00:00,No Simulation,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18
1,Audio_Moth_1,2025-03-18 16:59:21,no,0.0,No Bird Detected,0.000000,[],[],27.3,5.0,...,83,Mainly clear,2025-03-18 23:00:00+00:00,evening,2025-03-18 17:00:00,No Simulation,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18
2,Audio_Moth_1,2025-03-18 16:59:24,no,0.0,Myiornis atricapillus_Black-capped Pygmy-Tyrant,0.326591,[],[],27.3,5.0,...,83,Mainly clear,2025-03-18 23:00:00+00:00,evening,2025-03-18 17:00:00,No Simulation,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18
3,Audio_Moth_1,2025-03-18 16:59:27,no,0.0,No Bird Detected,0.000000,[],[],27.3,5.0,...,83,Mainly clear,2025-03-18 23:00:00+00:00,evening,2025-03-18 17:00:00,No Simulation,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18
4,Audio_Moth_1,2025-03-18 16:59:30,no,0.0,No Bird Detected,0.000000,[],[],27.3,5.0,...,83,Mainly clear,2025-03-18 23:00:00+00:00,evening,2025-03-18 17:00:00,No Simulation,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18


In [13]:
df_vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Recorder              10002 non-null  object 
 1   Timestamp             10002 non-null  object 
 2   Human Activity        10002 non-null  object 
 3   Human Activity Score  10002 non-null  float64
 4   species               10002 non-null  object 
 5   confidence            10002 non-null  float64
 6   Sim Type              10002 non-null  object 
 7   Sim Relative Time     10002 non-null  object 
 8   Temperature           10002 non-null  float64
 9   Windspeed             10002 non-null  float64
 10  Precipitation         10002 non-null  float64
 11  Humidity              10002 non-null  int64  
 12  Weather Desc          10002 non-null  object 
 13  Timestamp UTC         10002 non-null  object 
 14  Time Of Day           10002 non-null  object 
 15  Datetime           

#### Here is what we have about above data and how we gonna use it:


This dataset only includes the audio clips that fall within the three vehicle simulation windows — before, during, and after — across all AudioMoth recorders that vehicle events occured in.

It combines:

Core metadata: Recorder, Timestamp, species, and confidence, which describe when and where each detection occurred and which species was identified.

Simulation-related features: such as Sim_Start, Sim_End, period (window labels: before/during/after), and Sim_Type_Label, which define the context of each detection relative to the vehicle simulation event.

Environmental and contextual features: including weather variables (Temperature, Windspeed, Precipitation, Humidity, Weather Desc) and temporal information (Time of Day, Datetime, Sim Phase).


To use this dataset 

This vehicle-specific dataset will be used to analyze how vehicle simulation events impact bird species call rates.(we eventually want to figure out which speices are sensitive to vehicle simulation)

We will:

Quantify species detections before, during, and after vehicle events to measure changes in call rate patterns.

Identify potential sentinel species — those that consistently respond (increase or decrease in detections) during vehicle simulations across different recorders.

Add another layers, such as time of day and weather conditions, to understand how these environmental layers may influence or interact with species responses to vehicle simulations.


### Vehicle Simulation Impact Analysis Plan

In [14]:
## first let's count the species detections per period. 
## for each species and recorder I want to count the number of speices detections in each sim phase. 
## this will help us to understand the detection changes. 



species_counts = (
    df_vehicle
    .groupby(['Recorder', 'species', 'period'], as_index=False)
    .agg(detections=('species', 'size')) 
)

# pivot the table
species_pivot = (
    species_counts
    .pivot_table(
        index=['Recorder', 'species'],
        columns='period',
        values='detections',
        fill_value=0
    )
    .reset_index()
)






In [15]:
# let's check the result 

print(f"Number of unique species × recorder combinations: {len(species_pivot)}")
species_pivot.head(10)

Number of unique species × recorder combinations: 87


period,Recorder,species,after,before,during
0,Audio_Moth_1,Amazona farinosa_Mealy Parrot,1.0,0.0,0.0
1,Audio_Moth_1,Ara macao_Scarlet Macaw,0.0,11.0,0.0
2,Audio_Moth_1,Crypturellus soui_Little Tinamou,2.0,1.0,0.0
3,Audio_Moth_1,Cyanoloxia cyanoides_Blue-black Grosbeak,26.0,0.0,1.0
4,Audio_Moth_1,Formicarius analis_Black-faced Antthrush,1.0,0.0,0.0
5,Audio_Moth_1,Galbula ruficauda_Rufous-tailed Jacamar,0.0,2.0,0.0
6,Audio_Moth_1,Habia atrimaxillaris_Black-cheeked Ant-Tanager,1.0,0.0,0.0
7,Audio_Moth_1,Hylopezus perspicillatus_Streak-chested Antpitta,2.0,0.0,0.0
8,Audio_Moth_1,Hylophylax naevioides_Spotted Antbird,1.0,0.0,1.0
9,Audio_Moth_1,Lophotriccus pileatus_Scale-crested Pygmy-Tyrant,1.0,0.0,0.0


In [16]:
## now let's split above table by recorder for better viewing and analysis. 


recorder_groups = {
    name: group.reset_index(drop=True)
    for name, group in species_pivot.groupby('Recorder')
}

In [17]:
## this is audiomoth one

recorder_groups['Audio_Moth_1']


period,Recorder,species,after,before,during
0,Audio_Moth_1,Amazona farinosa_Mealy Parrot,1.0,0.0,0.0
1,Audio_Moth_1,Ara macao_Scarlet Macaw,0.0,11.0,0.0
2,Audio_Moth_1,Crypturellus soui_Little Tinamou,2.0,1.0,0.0
3,Audio_Moth_1,Cyanoloxia cyanoides_Blue-black Grosbeak,26.0,0.0,1.0
4,Audio_Moth_1,Formicarius analis_Black-faced Antthrush,1.0,0.0,0.0
5,Audio_Moth_1,Galbula ruficauda_Rufous-tailed Jacamar,0.0,2.0,0.0
6,Audio_Moth_1,Habia atrimaxillaris_Black-cheeked Ant-Tanager,1.0,0.0,0.0
7,Audio_Moth_1,Hylopezus perspicillatus_Streak-chested Antpitta,2.0,0.0,0.0
8,Audio_Moth_1,Hylophylax naevioides_Spotted Antbird,1.0,0.0,1.0
9,Audio_Moth_1,Lophotriccus pileatus_Scale-crested Pygmy-Tyrant,1.0,0.0,0.0


#### Key observations

1, the overall detections drop during the vehicle event. 
for example, “No Bird Detected” increased from 785 before  to 824 after, while bird detections overall are low during the simulation (many 0 in during)

this suggests that vehicle sound really strong that covered the bird calls or birds calls temporarily reduced

2, some speices were active before but completely silent during the vehicle event, like Ara macao, Microrhopias quixensis, Myiornis atricapillus

3, some speices show recovery after the event. such as: Myiornis atricapillus, Patagioenas nigrirostris

These show a pattern of delayed or rebound calling after vehicle noise subsides — possible sentinel-like response

4, many species show minimal detections overall, likely either rare or quiet speices in this location. 



In [18]:
recorder_groups['Audio_Moth_2']

period,Recorder,species,after,before,during
0,Audio_Moth_2,Arremon aurantiirostris_Orange-billed Sparrow,15.0,0.0,11.0
1,Audio_Moth_2,Baryphthengus martii_Rufous Motmot,2.0,2.0,0.0
2,Audio_Moth_2,Ceratopipra mentalis_Red-capped Manakin,9.0,4.0,1.0
3,Audio_Moth_2,Eucometis penicillata_Gray-headed Tanager,1.0,0.0,0.0
4,Audio_Moth_2,Geotrygon montana_Ruddy Quail-Dove,0.0,8.0,0.0
5,Audio_Moth_2,Habia atrimaxillaris_Black-cheeked Ant-Tanager,1.0,20.0,0.0
6,Audio_Moth_2,Hylopezus perspicillatus_Streak-chested Antpitta,2.0,9.0,0.0
7,Audio_Moth_2,Hylophylax naevioides_Spotted Antbird,4.0,0.0,0.0
8,Audio_Moth_2,Lepidothrix coronata_Blue-crowned Manakin,2.0,1.0,0.0
9,Audio_Moth_2,Leptodon cayanensis_Gray-headed Kite,0.0,0.0,4.0


#### key Observations

1, “No Bird Detected” increases from 236 before → 246 after, but especially notable is the 83 detections during, which is proportionally higher relative to the total clips (still indicating that many bird calls disappeared during the vehicle event).

2, same as recorder one, most species show 0 or near zero detections in the during. 

3, speices show strong before the event activity but silence during the event. 

such as Habia atrimaxillaris, Myiothlypis fulvicauda

These show complete silence during the vehicle, recovering weakly afterward — strong candidates for “sensitive” species.

4, some species show resilience. 

for example, Arremon aurantiirostris, this one actually increased after, showing potential rebound calling response after the event.

Ceratopipra mentalis, possible recovery or return of activity after the event. 

Leptodon cayanensis, only detected during the event, maybe this species less affected by noise. 


5, some speices like Myiornis atricapillus, Trogon massena, showed minimal but not zero activity during the event 




In [19]:
recorder_groups['Audio_Moth_4']

period,Recorder,species,after,before,during
0,Audio_Moth_4,Amazilia tzacatl_Rufous-tailed Hummingbird,0.0,2.0,0.0
1,Audio_Moth_4,Arremon aurantiirostris_Orange-billed Sparrow,7.0,0.0,0.0
2,Audio_Moth_4,Baryphthengus martii_Rufous Motmot,0.0,1.0,0.0
3,Audio_Moth_4,Cacicus uropygialis_Scarlet-rumped Cacique,4.0,0.0,0.0
4,Audio_Moth_4,Cantorchilus semibadius_Riverside Wren,0.0,0.0,2.0
5,Audio_Moth_4,Herpetotheres cachinnans_Laughing Falcon,0.0,2.0,0.0
6,Audio_Moth_4,Hylopezus perspicillatus_Streak-chested Antpitta,10.0,0.0,1.0
7,Audio_Moth_4,Lepidothrix coronata_Blue-crowned Manakin,5.0,4.0,0.0
8,Audio_Moth_4,Lipaugus unirufus_Rufous Piha,18.0,14.0,3.0
9,Audio_Moth_4,Myiobius sulphureipygius_Sulphur-rumped Flycat...,0.0,5.0,0.0


In [20]:
recorder_groups['Audio_Moth_5']

period,Recorder,species,after,before,during
0,Audio_Moth_5,Arremon aurantiirostris_Orange-billed Sparrow,36.0,124.0,0.0
1,Audio_Moth_5,Ceratopipra mentalis_Red-capped Manakin,1.0,1.0,0.0
2,Audio_Moth_5,Contopus virens_Eastern Wood-Pewee,1.0,0.0,0.0
3,Audio_Moth_5,Glyphorynchus spirurus_Wedge-billed Woodcreeper,1.0,0.0,0.0
4,Audio_Moth_5,Hylopezus perspicillatus_Streak-chested Antpitta,6.0,10.0,5.0
5,Audio_Moth_5,Malacoptila panamensis_White-whiskered Puffbird,0.0,1.0,1.0
6,Audio_Moth_5,Microrhopias quixensis_Dot-winged Antwren,0.0,1.0,1.0
7,Audio_Moth_5,Myiothlypis fulvicauda_Buff-rumped Warbler,7.0,12.0,5.0
8,Audio_Moth_5,No Bird Detected,267.0,759.0,141.0
9,Audio_Moth_5,Pachysylvia decurtata_Lesser Greenlet,0.0,0.0,1.0


In [21]:
recorder_groups['Audio_Moth_6']

period,Recorder,species,after,before,during
0,Audio_Moth_6,Ara macao_Scarlet Macaw,1.0,0.0,0.0
1,Audio_Moth_6,Arremon aurantiirostris_Orange-billed Sparrow,1.0,5.0,12.0
2,Audio_Moth_6,Cantorchilus semibadius_Riverside Wren,4.0,0.0,0.0
3,Audio_Moth_6,Ceratopipra mentalis_Red-capped Manakin,7.0,0.0,0.0
4,Audio_Moth_6,Chlorestes eliciae_Blue-throated Goldentail,3.0,24.0,0.0
5,Audio_Moth_6,Cyanoloxia cyanoides_Blue-black Grosbeak,5.0,0.0,0.0
6,Audio_Moth_6,Epinecrophylla fulviventris_Checker-throated S...,0.0,2.0,1.0
7,Audio_Moth_6,Eucometis penicillata_Gray-headed Tanager,0.0,0.0,1.0
8,Audio_Moth_6,Hylopezus perspicillatus_Streak-chested Antpitta,2.0,12.0,6.0
9,Audio_Moth_6,Hylophylax naevioides_Spotted Antbird,2.0,0.0,0.0


In [22]:
## now let's quantify the change across all the time windows 

## the below result will tell:
## Which species are most suppressed during vehicles,

## Which rebound the most after, and

## In how many recorders they were observed.


#compute the relative change metrics



species_pivot = species_pivot.copy()


species_pivot["before_during"] = (species_pivot["during"] - species_pivot["before"]) / (species_pivot["before"] + 1e-6)
species_pivot["before_after"]  = (species_pivot["after"]  - species_pivot["before"]) / (species_pivot["before"] + 1e-6)

#classify species responses per recorder
def classify_response(row):
    if row["before_during"] < -0.5:
        return "Suppressed_during"
    elif row["before_after"] > 0.5:
        return "Rebounded_after"
    elif abs(row["before_during"]) < 0.2 and abs(row["before_after"]) < 0.2:
        return "Unaffected"
    else:
        return "Mixed"

species_pivot["response_class"] = species_pivot.apply(classify_response, axis=1)


species_response_summary = (
    species_pivot
    .groupby("species")
    .agg({
        "before_during": "mean",
        "before_after": "mean",
        "Recorder": "nunique",
        "response_class": lambda x: x.value_counts().index[0] 
    })
    .rename(columns={"Recorder": "Num_Recorders"})
    .reset_index()
    .sort_values("before_during")
)

#identify sentinel candidates
species_response_summary["Sentinel_Type"] = np.select(
    [
        species_response_summary["before_during"] < -0.5,
        species_response_summary["before_after"] > 0.5
    ],
    [
        "Sensitive_Sentinel",
        "Resilient_Sentinel"
    ],
    default="Neutral"
)

# display

display(species_response_summary.head(20))


period,species,before_during,before_after,Num_Recorders,response_class,Sentinel_Type
8,Chlorestes eliciae_Blue-throated Goldentail,-1.000000,-0.875000,1,Suppressed_during,Sensitive_Sentinel
31,Myiornis atricapillus_Black-capped Pygmy-Tyrant,-1.000000,-0.505050,3,Suppressed_during,Sensitive_Sentinel
16,Geotrygon montana_Ruddy Quail-Dove,-1.000000,-1.000000,1,Suppressed_during,Sensitive_Sentinel
45,Thamnophilus doliatus_Barred Antshrike,-1.000000,-1.000000,1,Suppressed_during,Sensitive_Sentinel
30,Myiobius sulphureipygius_Sulphur-rumped Flycat...,-1.000000,-1.000000,1,Suppressed_during,Sensitive_Sentinel
38,Phaethornis striigularis_Stripe-throated Hermit,-1.000000,-1.000000,1,Suppressed_during,Sensitive_Sentinel
51,Tunchiornis ochraceiceps_Tawny-crowned Greenlet,-1.000000,-1.000000,2,Suppressed_during,Sensitive_Sentinel
19,Herpetotheres cachinnans_Laughing Falcon,-1.000000,-1.000000,1,Suppressed_during,Sensitive_Sentinel
15,Galbula ruficauda_Rufous-tailed Jacamar,-1.000000,-1.000000,1,Suppressed_during,Sensitive_Sentinel
0,Amazilia tzacatl_Rufous-tailed Hummingbird,-1.000000,-1.000000,1,Suppressed_during,Sensitive_Sentinel


In [23]:
## now let's split above table by the sentinel type and see which species in each type 


sensitive_sentinels = species_response_summary.query("Sentinel_Type == 'Sensitive_Sentinel'")
resilient_sentinels = species_response_summary.query("Sentinel_Type == 'Resilient_Sentinel'")
neutral_species     = species_response_summary.query("Sentinel_Type == 'Neutral'")

print(f"Sensitive sentinels: {len(sensitive_sentinels)} species")
print(f"Resilient sentinels: {len(resilient_sentinels)} species")
print(f"Neutral species: {len(neutral_species)} species")



Sensitive sentinels: 21 species
Resilient sentinels: 27 species
Neutral species: 6 species


In [24]:
##display

display(sensitive_sentinels.head(10))


period,species,before_during,before_after,Num_Recorders,response_class,Sentinel_Type
8,Chlorestes eliciae_Blue-throated Goldentail,-1.0,-0.87500,1,Suppressed_during,Sensitive_Sentinel
31,Myiornis atricapillus_Black-capped Pygmy-Tyrant,-1.0,-0.50505,3,Suppressed_during,Sensitive_Sentinel
16,Geotrygon montana_Ruddy Quail-Dove,-1.0,-1.00000,1,Suppressed_during,Sensitive_Sentinel
45,Thamnophilus doliatus_Barred Antshrike,-1.0,-1.00000,1,Suppressed_during,Sensitive_Sentinel
30,Myiobius sulphureipygius_Sulphur-rumped Flycat...,-1.0,-1.00000,1,Suppressed_during,Sensitive_Sentinel
38,Phaethornis striigularis_Stripe-throated Hermit,-1.0,-1.00000,1,Suppressed_during,Sensitive_Sentinel
51,Tunchiornis ochraceiceps_Tawny-crowned Greenlet,-1.0,-1.00000,2,Suppressed_during,Sensitive_Sentinel
19,Herpetotheres cachinnans_Laughing Falcon,-1.0,-1.00000,1,Suppressed_during,Sensitive_Sentinel
15,Galbula ruficauda_Rufous-tailed Jacamar,-1.0,-1.00000,1,Suppressed_during,Sensitive_Sentinel
0,Amazilia tzacatl_Rufous-tailed Hummingbird,-1.0,-1.00000,1,Suppressed_during,Sensitive_Sentinel


In [25]:

display(resilient_sentinels.head(10))


period,species,before_during,before_after,Num_Recorders,response_class,Sentinel_Type
18,Habia atrimaxillaris_Black-cheeked Ant-Tanager,-0.5000,4.999995e+05,2,Rebounded_after,Resilient_Sentinel
2,Ara macao_Scarlet Macaw,-0.5000,4.999995e+05,2,Suppressed_during,Resilient_Sentinel
46,Threnetes ruckeri_Band-tailed Barbthroat,-0.5000,1.500000e+06,2,Rebounded_after,Resilient_Sentinel
36,Patagioenas nigrirostris_Short-billed Pigeon,-0.1875,4.843745e+00,2,Mixed,Resilient_Sentinel
43,Seiurus aurocapilla_Ovenbird,0.0000,1.000000e+06,1,Rebounded_after,Resilient_Sentinel
41,Ramphastos ambiguus_Yellow-throated Toucan,0.0000,1.000000e+06,1,Rebounded_after,Resilient_Sentinel
39,Poliocrania exsul_Chestnut-backed Antbird,0.0000,1.000000e+06,1,Rebounded_after,Resilient_Sentinel
49,Trogon bairdii_Baird's Trogon,0.0000,2.000000e+06,1,Rebounded_after,Resilient_Sentinel
47,Tinamus major_Great Tinamou,0.0000,5.000000e+06,1,Rebounded_after,Resilient_Sentinel
40,Pyrilia haematotis_Brown-hooded Parrot,0.0000,2.000000e+06,1,Rebounded_after,Resilient_Sentinel


In [26]:

display(neutral_species)

period,species,before_during,before_after,Num_Recorders,response_class,Sentinel_Type
29,Microrhopias quixensis_Dot-winged Antwren,-4.999999e-01,-0.999999,2,Suppressed_during,Neutral
12,Epinecrophylla fulviventris_Checker-throated S...,-4.999998e-01,-1.000000,1,Mixed,Neutral
34,Pachysylvia decurtata_Lesser Greenlet,4.999995e+05,0.000000,2,Mixed,Neutral
35,Parkesia motacilla_Louisiana Waterthrush,4.999995e+05,-0.500000,2,Mixed,Neutral
52,Tyto alba_Barn Owl,1.000000e+06,0.000000,1,Mixed,Neutral
23,Leptodon cayanensis_Gray-headed Kite,4.000000e+06,0.000000,1,Mixed,Neutral


now that we've finished identifying species-level responses, let nove on to add layers. 

## Cross-context Analysis for vehicle Event 

Now that we know which species are affected, we’ll want to see how consistent those responses are across recorders, days, and environments.

#### Recorder Level Consistency 

Question we need to answer is: Do species that are classified as sensitive or resilient show the same response patterns across different audiomoth recorders (location)?




In [ ]:
## we start from sensitive_sentinels table. 

## I want to check if a species (sensitive or resilient) are detected in 5, 4, 3, 2, or only one recorder. 

## let's start with the sensitive sentinels. 


## let's filter out the species that presents in more than one recorder. 


multi_recorder_sensitives = sensitive_sentinels[sensitive_sentinels["Num_Recorders"] > 1]
display(multi_recorder_sensitives)


period,species,before_during,before_after,Num_Recorders,response_class,Sentinel_Type
31,Myiornis atricapillus_Black-capped Pygmy-Tyrant,-1.000000,-5.050505e-01,3,Suppressed_during,Sensitive_Sentinel
51,Tunchiornis ochraceiceps_Tawny-crowned Greenlet,-1.000000,-9.999996e-01,2,Suppressed_during,Sensitive_Sentinel
22,Lepidothrix coronata_Blue-crowned Manakin,-0.999999,6.249995e-01,2,Suppressed_during,Sensitive_Sentinel
4,Baryphthengus martii_Rufous Motmot,-0.999999,-4.999995e-01,2,Suppressed_during,Sensitive_Sentinel
32,Myiothlypis fulvicauda_Buff-rumped Warbler,-0.837302,-2.685185e-01,3,Suppressed_during,Sensitive_Sentinel
33,No Bird Detected,-0.728563,-1.500768e-01,5,Suppressed_during,Sensitive_Sentinel
7,Ceratopipra mentalis_Red-capped Manakin,-0.583333,2.333334e+06,3,Suppressed_during,Sensitive_Sentinel


I guess it's not superised that no species is detected across all 5 recorders. 

In [41]:
## I just want to check what are the recorders that the species detected in more than just one recorder in 


multi_recorder_map_sensitive = (
    species_pivot[species_pivot["species"].isin(sensitive_sentinels[ sensitive_sentinels["Num_Recorders"] > 1 ]["species"])]
    .groupby("species")["Recorder"]
    .unique()
    .reset_index()
)
display(multi_recorder_map_sensitive)

,species,Recorder
0,Baryphthengus martii_Rufous Motmot,"[Audio_Moth_2, Audio_Moth_4]"
1,Ceratopipra mentalis_Red-capped Manakin,"[Audio_Moth_2, Audio_Moth_5, Audio_Moth_6]"
2,Lepidothrix coronata_Blue-crowned Manakin,"[Audio_Moth_2, Audio_Moth_4]"
3,Myiornis atricapillus_Black-capped Pygmy-Tyrant,"[Audio_Moth_1, Audio_Moth_2, Audio_Moth_6]"
4,Myiothlypis fulvicauda_Buff-rumped Warbler,"[Audio_Moth_2, Audio_Moth_5, Audio_Moth_6]"
5,No Bird Detected,"[Audio_Moth_1, Audio_Moth_2, Audio_Moth_4, Aud..."
6,Tunchiornis ochraceiceps_Tawny-crowned Greenlet,"[Audio_Moth_2, Audio_Moth_4]"


In [36]:
## do the same to the resilient species. 
multi_recorder_resilents = resilient_sentinels[resilient_sentinels["Num_Recorders"] > 1]
display(multi_recorder_resilents)


period,species,before_during,before_after,Num_Recorders,response_class,Sentinel_Type
18,Habia atrimaxillaris_Black-cheeked Ant-Tanager,-5.000000e-01,4.999995e+05,2,Rebounded_after,Resilient_Sentinel
2,Ara macao_Scarlet Macaw,-5.000000e-01,4.999995e+05,2,Suppressed_during,Resilient_Sentinel
46,Threnetes ruckeri_Band-tailed Barbthroat,-4.999995e-01,1.500000e+06,2,Rebounded_after,Resilient_Sentinel
36,Patagioenas nigrirostris_Short-billed Pigeon,-1.874995e-01,4.843745e+00,2,Mixed,Resilient_Sentinel
26,Malacoptila panamensis_White-whiskered Puffbird,0.000000e+00,4.999995e+05,2,Rebounded_after,Resilient_Sentinel
20,Hylopezus perspicillatus_Streak-chested Antpitta,1.999996e+05,2.400000e+06,5,Rebounded_after,Resilient_Sentinel
21,Hylophylax naevioides_Spotted Antbird,3.333333e+05,2.333333e+06,3,Rebounded_after,Resilient_Sentinel
11,Cyanoloxia cyanoides_Blue-black Grosbeak,5.000000e+05,1.550000e+07,2,Rebounded_after,Resilient_Sentinel
13,Eucometis penicillata_Gray-headed Tanager,5.000000e+05,5.000000e+05,2,Rebounded_after,Resilient_Sentinel
6,Cantorchilus semibadius_Riverside Wren,1.000000e+06,2.000000e+06,2,Mixed,Resilient_Sentinel


we got one resilent species that detected in all 5 recorders. 

In [40]:
multi_recorder_map_resilient = (
    species_pivot[species_pivot["species"].isin(resilient_sentinels[ resilient_sentinels["Num_Recorders"] > 1 ]["species"])]
    .groupby("species")["Recorder"]
    .unique()
    .reset_index()
)
display(multi_recorder_map_resilient)

,species,Recorder
0,Ara macao_Scarlet Macaw,"[Audio_Moth_1, Audio_Moth_6]"
1,Arremon aurantiirostris_Orange-billed Sparrow,"[Audio_Moth_2, Audio_Moth_4, Audio_Moth_5, Aud..."
2,Cantorchilus semibadius_Riverside Wren,"[Audio_Moth_4, Audio_Moth_6]"
3,Cyanoloxia cyanoides_Blue-black Grosbeak,"[Audio_Moth_1, Audio_Moth_6]"
4,Eucometis penicillata_Gray-headed Tanager,"[Audio_Moth_2, Audio_Moth_6]"
5,Habia atrimaxillaris_Black-cheeked Ant-Tanager,"[Audio_Moth_1, Audio_Moth_2]"
6,Hylopezus perspicillatus_Streak-chested Antpitta,"[Audio_Moth_1, Audio_Moth_2, Audio_Moth_4, Aud..."
7,Hylophylax naevioides_Spotted Antbird,"[Audio_Moth_1, Audio_Moth_2, Audio_Moth_6]"
8,Malacoptila panamensis_White-whiskered Puffbird,"[Audio_Moth_1, Audio_Moth_5]"
9,Patagioenas nigrirostris_Short-billed Pigeon,"[Audio_Moth_1, Audio_Moth_2]"


#### Time of Day Effects

so,to add this time of day layer, I think my question would be: for species that are sensitive or resilient to vehical simulation, how does the time of day influence their call activity patterns?

In [44]:
cols_to_keep = [
    "Recorder", 
    "species", 
    "confidence", 
    "Timestamp", 
    "Time Of Day", 
    "period", 
    "Sim_Type_Label", 
    "Sim_Start", 
    "Sim_End"
]

df_vehicle_time = df_vehicle[cols_to_keep].copy()

display(df_vehicle_time.head())


,Recorder,species,confidence,Timestamp,Time Of Day,period,Sim_Type_Label,Sim_Start,Sim_End
0,Audio_Moth_1,No Bird Detected,0.000000,2025-03-18 16:59:18,evening,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18
1,Audio_Moth_1,No Bird Detected,0.000000,2025-03-18 16:59:21,evening,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18
2,Audio_Moth_1,Myiornis atricapillus_Black-capped Pygmy-Tyrant,0.326591,2025-03-18 16:59:24,evening,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18
3,Audio_Moth_1,No Bird Detected,0.000000,2025-03-18 16:59:27,evening,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18
4,Audio_Moth_1,No Bird Detected,0.000000,2025-03-18 16:59:30,evening,before,Vehicle,2025-03-18 17:14:18,2025-03-18 17:19:18


In [47]:
## first I just want to see the detection situation in each recorder at different time of day--regardless of the sim. 

# group by recorder,  time of day and species
detections_by_species_time = (
    df_vehicle_time
    .groupby(["Recorder", "Time Of Day", "species"])
    .size()
    .reset_index(name="detections")
    .sort_values(["Recorder", "Time Of Day", "detections"], ascending=[True, True, False])
)

display(detections_by_species_time.head(20))



,Recorder,Time Of Day,species,detections
10,Audio_Moth_1,evening,No Bird Detected,1361
1,Audio_Moth_1,evening,Ara macao_Scarlet Macaw,11
9,Audio_Moth_1,evening,Myiornis atricapillus_Black-capped Pygmy-Tyrant,11
8,Audio_Moth_1,evening,Microrhopias quixensis_Dot-winged Antwren,5
12,Audio_Moth_1,evening,Tinamus major_Great Tinamou,5
2,Audio_Moth_1,evening,Crypturellus soui_Little Tinamou,2
6,Audio_Moth_1,evening,Hylopezus perspicillatus_Streak-chested Antpitta,2
13,Audio_Moth_1,evening,Xiphorhynchus susurrans_Cocoa Woodcreeper,2
0,Audio_Moth_1,evening,Amazona farinosa_Mealy Parrot,1
3,Audio_Moth_1,evening,Formicarius analis_Black-faced Antthrush,1


In [49]:
## now I want to split into each recorder table

# Split into individual recorder tables
recorder_groups_time = {
    recorder: group.reset_index(drop=True)
    for recorder, group in detections_by_species_time.groupby("Recorder")
}



In [51]:
## let's show the audiomoth one

rec1 = recorder_groups_time['Audio_Moth_1']

rec1


,Recorder,Time Of Day,species,detections
0,Audio_Moth_1,evening,No Bird Detected,1361
1,Audio_Moth_1,evening,Ara macao_Scarlet Macaw,11
2,Audio_Moth_1,evening,Myiornis atricapillus_Black-capped Pygmy-Tyrant,11
3,Audio_Moth_1,evening,Microrhopias quixensis_Dot-winged Antwren,5
4,Audio_Moth_1,evening,Tinamus major_Great Tinamou,5
5,Audio_Moth_1,evening,Crypturellus soui_Little Tinamou,2
6,Audio_Moth_1,evening,Hylopezus perspicillatus_Streak-chested Antpitta,2
7,Audio_Moth_1,evening,Xiphorhynchus susurrans_Cocoa Woodcreeper,2
8,Audio_Moth_1,evening,Amazona farinosa_Mealy Parrot,1
9,Audio_Moth_1,evening,Formicarius analis_Black-faced Antthrush,1


In [53]:
rec2 = recorder_groups_time['Audio_Moth_2']

rec2

,Recorder,Time Of Day,species,detections
0,Audio_Moth_2,morning,No Bird Detected,565
1,Audio_Moth_2,morning,Arremon aurantiirostris_Orange-billed Sparrow,26
2,Audio_Moth_2,morning,Habia atrimaxillaris_Black-cheeked Ant-Tanager,21
3,Audio_Moth_2,morning,Myiothlypis fulvicauda_Buff-rumped Warbler,15
4,Audio_Moth_2,morning,Ceratopipra mentalis_Red-capped Manakin,14
5,Audio_Moth_2,morning,Patagioenas nigrirostris_Short-billed Pigeon,12
6,Audio_Moth_2,morning,Hylopezus perspicillatus_Streak-chested Antpitta,11
7,Audio_Moth_2,morning,Myiornis atricapillus_Black-capped Pygmy-Tyrant,10
8,Audio_Moth_2,morning,Geotrygon montana_Ruddy Quail-Dove,8
9,Audio_Moth_2,morning,Trogon massena_Slaty-tailed Trogon,7


In [54]:
rec4 = recorder_groups_time['Audio_Moth_4']

rec4

,Recorder,Time Of Day,species,detections
0,Audio_Moth_4,afternoon,No Bird Detected,379
1,Audio_Moth_4,afternoon,Tyto alba_Barn Owl,1
2,Audio_Moth_4,evening,No Bird Detected,674
3,Audio_Moth_4,evening,Myiobius sulphureipygius_Sulphur-rumped Flycat...,5
4,Audio_Moth_4,evening,Baryphthengus martii_Rufous Motmot,1
5,Audio_Moth_4,morning,No Bird Detected,1931
6,Audio_Moth_4,morning,Lipaugus unirufus_Rufous Piha,35
7,Audio_Moth_4,morning,Hylopezus perspicillatus_Streak-chested Antpitta,11
8,Audio_Moth_4,morning,Lepidothrix coronata_Blue-crowned Manakin,9
9,Audio_Moth_4,morning,Arremon aurantiirostris_Orange-billed Sparrow,7


In [55]:
rec5 = recorder_groups_time['Audio_Moth_5']

rec5

,Recorder,Time Of Day,species,detections
0,Audio_Moth_5,afternoon,No Bird Detected,543
1,Audio_Moth_5,afternoon,Arremon aurantiirostris_Orange-billed Sparrow,124
2,Audio_Moth_5,afternoon,Myiothlypis fulvicauda_Buff-rumped Warbler,12
3,Audio_Moth_5,morning,No Bird Detected,624
4,Audio_Moth_5,morning,Arremon aurantiirostris_Orange-billed Sparrow,36
5,Audio_Moth_5,morning,Hylopezus perspicillatus_Streak-chested Antpitta,21
6,Audio_Moth_5,morning,Myiothlypis fulvicauda_Buff-rumped Warbler,12
7,Audio_Moth_5,morning,Threnetes ruckeri_Band-tailed Barbthroat,3
8,Audio_Moth_5,morning,Ceratopipra mentalis_Red-capped Manakin,2
9,Audio_Moth_5,morning,Malacoptila panamensis_White-whiskered Puffbird,2


In [56]:
rec6 = recorder_groups_time['Audio_Moth_6']

rec6

,Recorder,Time Of Day,species,detections
0,Audio_Moth_6,evening,No Bird Detected,645
1,Audio_Moth_6,evening,Myiothlypis fulvicauda_Buff-rumped Warbler,36
2,Audio_Moth_6,evening,Myiornis atricapillus_Black-capped Pygmy-Tyrant,12
3,Audio_Moth_6,evening,Arremon aurantiirostris_Orange-billed Sparrow,11
4,Audio_Moth_6,evening,Epinecrophylla fulviventris_Checker-throated S...,1
5,Audio_Moth_6,evening,Eucometis penicillata_Gray-headed Tanager,1
6,Audio_Moth_6,evening,Parkesia motacilla_Louisiana Waterthrush,1
7,Audio_Moth_6,morning,No Bird Detected,1863
8,Audio_Moth_6,morning,Myiothlypis fulvicauda_Buff-rumped Warbler,58
9,Audio_Moth_6,morning,Chlorestes eliciae_Blue-throated Goldentail,27


In [58]:
## I want to split the detections_by_species_time by the time of day. 
## I think I will do the time of day type (morning, afternoon, evening) one by one across all the recorders. 
## from above analysis we can see that not all the recorders have all three types of time of day. most of them only have two of them. 

tod_groups_time = {
    tod: group.reset_index(drop=True)
    for tod, group in detections_by_species_time.groupby("Time Of Day")
}

In [ ]:
morning = tod_groups_time['morning']
morning

,Recorder,Time Of Day,species,detections
0,Audio_Moth_1,morning,No Bird Detected,490
1,Audio_Moth_1,morning,Patagioenas nigrirostris_Short-billed Pigeon,106
2,Audio_Moth_1,morning,Thamnophilus bridgesi_Black-hooded Antshrike,70
3,Audio_Moth_1,morning,Cyanoloxia cyanoides_Blue-black Grosbeak,27
4,Audio_Moth_1,morning,Thamnophilus doliatus_Barred Antshrike,6
...,...,...,...,...
67,Audio_Moth_6,morning,Epinecrophylla fulviventris_Checker-throated S...,2
68,Audio_Moth_6,morning,Hylophylax naevioides_Spotted Antbird,2
69,Audio_Moth_6,morning,Ara macao_Scarlet Macaw,1
70,Audio_Moth_6,morning,Saltator maximus_Buff-throated Saltator,1


In [61]:
## I want to see only the species that detected more than 10 times in the morning. 

morning_10 = morning[morning['detections'] > 10]
morning_10

,Recorder,Time Of Day,species,detections
0,Audio_Moth_1,morning,No Bird Detected,490
1,Audio_Moth_1,morning,Patagioenas nigrirostris_Short-billed Pigeon,106
2,Audio_Moth_1,morning,Thamnophilus bridgesi_Black-hooded Antshrike,70
3,Audio_Moth_1,morning,Cyanoloxia cyanoides_Blue-black Grosbeak,27
13,Audio_Moth_2,morning,No Bird Detected,565
14,Audio_Moth_2,morning,Arremon aurantiirostris_Orange-billed Sparrow,26
15,Audio_Moth_2,morning,Habia atrimaxillaris_Black-cheeked Ant-Tanager,21
16,Audio_Moth_2,morning,Myiothlypis fulvicauda_Buff-rumped Warbler,15
17,Audio_Moth_2,morning,Ceratopipra mentalis_Red-capped Manakin,14
18,Audio_Moth_2,morning,Patagioenas nigrirostris_Short-billed Pigeon,12


In [64]:
evening = tod_groups_time['evening']
evening_10 = evening[evening['detections'] > 10]

evening

,Recorder,Time Of Day,species,detections
0,Audio_Moth_1,evening,No Bird Detected,1361
1,Audio_Moth_1,evening,Ara macao_Scarlet Macaw,11
2,Audio_Moth_1,evening,Myiornis atricapillus_Black-capped Pygmy-Tyrant,11
3,Audio_Moth_1,evening,Microrhopias quixensis_Dot-winged Antwren,5
4,Audio_Moth_1,evening,Tinamus major_Great Tinamou,5
5,Audio_Moth_1,evening,Crypturellus soui_Little Tinamou,2
6,Audio_Moth_1,evening,Hylopezus perspicillatus_Streak-chested Antpitta,2
7,Audio_Moth_1,evening,Xiphorhynchus susurrans_Cocoa Woodcreeper,2
8,Audio_Moth_1,evening,Amazona farinosa_Mealy Parrot,1
9,Audio_Moth_1,evening,Formicarius analis_Black-faced Antthrush,1


In [65]:
evening_10

,Recorder,Time Of Day,species,detections
0,Audio_Moth_1,evening,No Bird Detected,1361
1,Audio_Moth_1,evening,Ara macao_Scarlet Macaw,11
2,Audio_Moth_1,evening,Myiornis atricapillus_Black-capped Pygmy-Tyrant,11
14,Audio_Moth_4,evening,No Bird Detected,674
17,Audio_Moth_6,evening,No Bird Detected,645
18,Audio_Moth_6,evening,Myiothlypis fulvicauda_Buff-rumped Warbler,36
19,Audio_Moth_6,evening,Myiornis atricapillus_Black-capped Pygmy-Tyrant,12
20,Audio_Moth_6,evening,Arremon aurantiirostris_Orange-billed Sparrow,11


In [66]:
afternoon = tod_groups_time['afternoon']
afternoon_10 = afternoon[afternoon['detections'] > 10]

afternoon

,Recorder,Time Of Day,species,detections
0,Audio_Moth_4,afternoon,No Bird Detected,379
1,Audio_Moth_4,afternoon,Tyto alba_Barn Owl,1
2,Audio_Moth_5,afternoon,No Bird Detected,543
3,Audio_Moth_5,afternoon,Arremon aurantiirostris_Orange-billed Sparrow,124
4,Audio_Moth_5,afternoon,Myiothlypis fulvicauda_Buff-rumped Warbler,12


#### Weather Effects 